In [2]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd
import os
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU not found. Using CPU instead.')

# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = tf.convert_to_tensor(x_train.reshape(-1, 784).astype(np.float32) / 255.0)
x_test = tf.convert_to_tensor(x_test.reshape(-1, 784).astype(np.float32) / 255.0)

y_train = tf.one_hot(y_train, depth=10, dtype=tf.float32)
y_test_labels = tf.convert_to_tensor(y_test)  # Original labels for confusion matrix
y_test = tf.one_hot(y_test, depth=10, dtype=tf.float32)

# Model parameters
n_hidden = 256
tf.random.set_seed(42)

W1 = tf.Variable(tf.random.normal([784, n_hidden], stddev=0.1, dtype=tf.float32))
b1 = tf.Variable(tf.random.normal([n_hidden], stddev=0.1, dtype=tf.float32))
W2 = tf.Variable(tf.random.normal([n_hidden, 10], stddev=0.1, dtype=tf.float32))
b2 = tf.Variable(tf.random.normal([10], stddev=0.1, dtype=tf.float32))

# Define model
@tf.function
def model(X):
    Z1 = tf.add(tf.matmul(X, W1), b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(A1, W2), b2)
    return Z2

# Loss function
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

# Optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.1)

# Training step
@tf.function
def train_step(batch_x, batch_y):
    with tf.GradientTape() as tape:
        logits = model(batch_x)
        loss = loss_fn(batch_y, logits)
    grads = tape.gradient(loss, [W1, b1, W2, b2])
    optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2]))
    return loss

# Configurations for batch size and epochs
configs = [(1000, 50)]

# Excel file setup
output_file = "fashion_mnist_nn_results_1000.xlsx"

# If file exists, load; else, create a new one
if os.path.exists(output_file):
    try:
        wb = load_workbook(output_file)
        ws = wb.active
    except:
        os.remove(output_file)  # Delete corrupted file
        wb = Workbook()
        ws = wb.active
        ws.title = "Results"
        ws.append(["Batch Size", "Epochs", "Train Loss", "Train Accuracy", "Test Accuracy", "Execution Time"])
        wb.save(output_file)
else:
    wb = Workbook()
    ws = wb.active
    ws.title = "Results"
    ws.append(["Batch Size", "Epochs", "Train Loss", "Train Accuracy", "Test Accuracy", "Execution Time"])
    wb.save(output_file)

# Run training
with tf.device('/GPU:0'):
    for batch_size, epochs in configs:
        start_time = time.time()
        loss_curve, acc_curve = [], []

        dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size).shuffle(10000).prefetch(tf.data.AUTOTUNE)

        for epoch in range(epochs):
            for batch_x, batch_y in dataset:
                train_loss = train_step(batch_x, batch_y)

            # Compute train accuracy
            train_logits = model(x_train)
            train_acc = tf.reduce_mean(tf.cast(tf.argmax(train_logits, axis=1) == tf.argmax(y_train, axis=1), tf.float32))
            loss_curve.append(train_loss.numpy())
            acc_curve.append(train_acc.numpy())

        # Compute test accuracy
        test_logits = model(x_test)
        test_acc = tf.reduce_mean(tf.cast(tf.argmax(test_logits, axis=1) == tf.argmax(y_test, axis=1), tf.float32))
        y_pred = tf.argmax(test_logits, axis=1)
        conf_matrix = confusion_matrix(y_test_labels.numpy(), y_pred.numpy())
        exec_time = time.time() - start_time

        # Append results to Excel
        ws.append([batch_size, epochs, train_loss.numpy(), train_acc.numpy(), test_acc.numpy(), exec_time])

        # Save intermediate results
        wb.save(output_file)

        # Plot Loss and Accuracy Curves
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(range(epochs), loss_curve, label='Loss', color='red')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Loss Curve (Batch {batch_size}, Epochs {epochs})')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(range(epochs), acc_curve, label='Accuracy', color='blue')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title(f'Accuracy Curve (Batch {batch_size}, Epochs {epochs})')
        plt.legend()

        plt.tight_layout()
        plt.savefig(f"fashion_mnist_curves_{batch_size}_{epochs}.png")
        plt.close()

        # Save confusion matrix
        df_conf_matrix = pd.DataFrame(conf_matrix)
        with pd.ExcelWriter(output_file, mode='a', if_sheet_exists="new") as writer:
            df_conf_matrix.to_excel(writer, sheet_name=f"Conf_Matrix_{batch_size}_{epochs}")

        # Add accuracy/loss curve image to Excel
        ws_image = wb.create_sheet(title=f"Curve_{batch_size}_{epochs}")
        img = Image(f"fashion_mnist_curves_{batch_size}_{epochs}.png")
        ws_image.add_image(img, "A1")

        print(f"Batch Size: {batch_size}, Epochs: {epochs}")
        print(f"Test Accuracy: {test_acc.numpy():.4f}")
        print(f"Execution Time: {exec_time:.2f} seconds\n")

    # Final Save
    wb.save(output_file)
    wb.close()

print(f"Results saved to {output_file}")


Num GPUs Available:  0
GPU not found. Using CPU instead.
Batch Size: 1000, Epochs: 50
Test Accuracy: 0.8198
Execution Time: 34.45 seconds

Results saved to fashion_mnist_nn_results_1000.xlsx
